In [ ]:
# default_exp exec.train_tacotron2

In [ ]:
# export
from uberduck_ml_dev.trainer.tacotron2 import Tacotron2Trainer
from uberduck_ml_dev.vendor.tfcompat.hparam import HParams
from uberduck_ml_dev.trainer.tacotron2 import DEFAULTS as TACOTRON2_TRAINER_DEFAULTS
import argparse
import sys
import json
import torch
from torch import multiprocessing as mp

In [ ]:
# export
def parse_args(args):
    parser = argparse.ArgumentParser()
    parser.add_argument("--config", help="Path to JSON config")
    args = parser.parse_args(args)
    return args


def run(rank, device_count, hparams):
    trainer = Tacotron2Trainer(hparams, rank=rank, world_size=device_count)
    try:
        trainer.train()
    except Exception as e:
        print(f"Exception raised while training: {e}")
        # TODO: save state.
        raise e

In [ ]:
# export
try:
    from nbdev.imports import IN_NOTEBOOK
except:
    IN_NOTEBOOK = False
if __name__ == "__main__" and not IN_NOTEBOOK:
    args = parse_args(sys.argv[1:])
    config = TACOTRON2_TRAINER_DEFAULTS.values()
    if args.config:
        with open(args.config) as f:
            config.update(json.load(f))
    config.update(vars(args))
    hparams = HParams(**config)
    try:
        if hparams.distributed_run:
            device_count = torch.cuda.device_count()
            mp.spawn(run, (device_count, hparams), device_count)
        else:
            run(None, None, hparams)
    except KeyboardInterrupt:
        print("Graceful shutdown, saving current checkpoint")

In [ ]:
# skip
config = TACOTRON2_TRAINER_DEFAULTS.values()
with open("test/fixtures/ljtest/taco2_lj2lj.json") as f:
    config.update(json.load(f))
hparams = HParams(**config)
print(hparams)
try:
    if hparams.distributed_run:
        device_count = torch.cuda.device_count()
        mp.spawn(run, (device_count, hparams), device_count)
    else:
        run(None, None, hparams)
except KeyboardInterrupt:
    print("Graceful shutdown, saving current checkpoint")

[('attention_dim', 128), ('attention_location_kernel_size', 31), ('attention_location_n_filters', 32), ('attention_rnn_dim', 1024), ('batch_size', 16), ('checkpoint_name', None), ('checkpoint_path', 'test/fixtures/results/checkpoints'), ('coarse_n_frames_per_step', None), ('cudnn_enabled', True), ('dataset_path', './dataset'), ('debug', False), ('decay_rate', 8000), ('decay_start', 15000), ('decoder_rnn_dim', 1024), ('distributed_run', False), ('encoder_embedding_dim', 512), ('encoder_kernel_size', 5), ('encoder_n_convolutions', 3), ('epochs', 5), ('epochs_per_checkpoint', 4), ('filter_length', 1024), ('fp16_run', False), ('gate_threshold', 0.5), ('grad_clip_thresh', 1.0), ('gst_type', None), ('has_speaker_embedding', False), ('hop_length', 256), ('ignore_layers', ['speaker_embedding.weight']), ('include_f0', False), ('is_validate', True), ('learning_rate', 0.001), ('log_dir', 'test/fixtures/results/logs'), ('mask_padding', True), ('max_decoder_steps', 1000), ('max_wav_value', 32768.0)